In [1]:
import numpy as np
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from ema_workbench.util import ema_logging
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench import save_results, load_results
from ema_workbench import (Model, RealParameter, perform_experiments, ScalarOutcome)

from ema_workbench import (
    Model,
    MultiprocessingEvaluator,
    ScalarOutcome,
    IntegerParameter,
    optimize,
    Scenario,
)
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench.util import ema_logging

from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [3]:
# Creating the policy dictionary based on the image data
policy_dict = {
    '0_RfR 0': 1,
    '0_RfR 1': 1,
    '0_RfR 2': 1,
    '1_RfR 0': 0,
    '1_RfR 1': 0,
    '1_RfR 2': 0,
    '2_RfR 0': 1,
    '2_RfR 1': 1,
    '2_RfR 2': 1,
    '3_RfR 0': 1,
    '3_RfR 1': 1,
    '3_RfR 2': 1,
    '4_RfR 0': 1,
    '4_RfR 1': 0,
    '4_RfR 2': 0,
    'EWS_DaysToThreat': 3,
    'A.1_DikeIncrease 0': 0,
    'A.1_DikeIncrease 1': 0,
    'A.1_DikeIncrease 2': 0,
    'A.2_DikeIncrease 0': 0,
    'A.2_DikeIncrease 1': 0,
    'A.2_DikeIncrease 2': 0,
    'A.3_DikeIncrease 0': 10,
    'A.3_DikeIncrease 1': 0,
    'A.3_DikeIncrease 2': 0,
    'A.4_DikeIncrease 0': 1,
    'A.4_DikeIncrease 1': 0,
    'A.4_DikeIncrease 2': 0,
    'A.5_DikeIncrease 0': 0,
    'A.5_DikeIncrease 1': 0,
    'A.5_DikeIncrease 2': 0,
}

# Creating the policy using the Policy class
policy = Policy('worst_policy', **policy_dict)

# Display the policy
print(policy)

Policy({'0_RfR 0': 1, '0_RfR 1': 1, '0_RfR 2': 1, '1_RfR 0': 0, '1_RfR 1': 0, '1_RfR 2': 0, '2_RfR 0': 1, '2_RfR 1': 1, '2_RfR 2': 1, '3_RfR 0': 1, '3_RfR 1': 1, '3_RfR 2': 1, '4_RfR 0': 1, '4_RfR 1': 0, '4_RfR 2': 0, 'EWS_DaysToThreat': 3, 'A.1_DikeIncrease 0': 0, 'A.1_DikeIncrease 1': 0, 'A.1_DikeIncrease 2': 0, 'A.2_DikeIncrease 0': 0, 'A.2_DikeIncrease 1': 0, 'A.2_DikeIncrease 2': 0, 'A.3_DikeIncrease 0': 10, 'A.3_DikeIncrease 1': 0, 'A.3_DikeIncrease 2': 0, 'A.4_DikeIncrease 0': 1, 'A.4_DikeIncrease 1': 0, 'A.4_DikeIncrease 2': 0, 'A.5_DikeIncrease 0': 0, 'A.5_DikeIncrease 1': 0, 'A.5_DikeIncrease 2': 0})


In [4]:
n_scenarios = 15000
with MultiprocessingEvaluator(dike_model, n_processes=-2) as evaluator:
    results_policies_prim = evaluator.perform_experiments(n_scenarios,
                                            policy)

[MainProcess/INFO] pool started with 14 workers
[MainProcess/INFO] performing 15000 scenarios * 1 policies * 1 model(s) = 15000 experiments
100%|████████████████████████████████████| 15000/15000 [10:18<00:00, 24.25it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [10]:
from ema_workbench.analysis import prim
experiments, outcomes = results_policies_prim

x = experiments.drop(columns=['0_RfR 0',
                          '0_RfR 1',
                          '0_RfR 2',
                          '1_RfR 0',
                          '1_RfR 1',
                          '1_RfR 2',
                          '2_RfR 0',
                          '2_RfR 1',
                          '2_RfR 2',
                          '3_RfR 0',
                          '3_RfR 1',
                          '3_RfR 2',
                          '4_RfR 0',
                          '4_RfR 1',
                          '4_RfR 2',
                          'EWS_DaysToThreat',
                          'A.1_DikeIncrease 0',
                          'A.1_DikeIncrease 1',
                          'A.1_DikeIncrease 2',
                          'A.2_DikeIncrease 0',
                          'A.2_DikeIncrease 1',
                          'A.2_DikeIncrease 2',
                          'A.3_DikeIncrease 0',
                          'A.3_DikeIncrease 1',
                          'A.3_DikeIncrease 2',
                          'A.4_DikeIncrease 0',
                          'A.4_DikeIncrease 1',
                          'A.4_DikeIncrease 2',
                          'A.5_DikeIncrease 0',
                          'A.5_DikeIncrease 1',
                          'A.5_DikeIncrease 2',
                          'policy'
                          ])
# y = outcomes['RfR Investment Costs'] < 0.5

y = outcomes['Expected Annual Damage'] > np.percentile(outcomes['Expected Annual Damage'], 90)
prim_alg = prim.Prim(x, y, threshold=0.8)
box = prim_alg.find_box()

[MainProcess/INFO] column model dropped from analysis because it has only one category
[MainProcess/INFO] 15000 points remaining, containing 1500 cases of interest
[MainProcess/INFO] mean: 0.9880952380952381, mass: 0.0504, coverage: 0.498, density: 0.9880952380952381 restricted_dimensions: 1


In [11]:
box.inspect_tradeoff()

alt.VConcatChart(...)

In [7]:
y = outcomes['Expected Number of Deaths'] > np.percentile(outcomes['Expected Number of Deaths'], 90)
prim_alg = prim.Prim(x, y, threshold=0.8)
box = prim_alg.find_box()
box.inspect_tradeoff()

[MainProcess/INFO] column model dropped from analysis because it has only one category
[MainProcess/INFO] 15000 points remaining, containing 1500 cases of interest
[MainProcess/INFO] mean: 1.0, mass: 0.05586666666666667, coverage: 0.5586666666666666, density: 1.0 restricted_dimensions: 1


alt.VConcatChart(...)